In [1]:
from os import listdir
from os.path import isfile, join

import numpy as np
import pandas as pd
import argparse
import random
import uuid
import pickle
import itertools

from labelisation import labelisation_automatic
from CompositionTree_RCIS import composition_tree


In [2]:
kernel_size = 1
kernel_stride = 1
nbsplitlabel = 4
window = 5
dirdataset = "./campagne_sge/Train_DataSet/"
#dirdataset= "C:/Users/IBK/Desktop/2emecontribution/ConsommationSGE/ALL"
list_file = [f for f in listdir(dirdataset) if isfile(join(dirdataset, f))]
step = 1
offset = 1
mind = (window-1)/2

features = []
classes = []
fclasses = []
values = []
labels = []
nb_observation = 0

for i, f in enumerate(list_file):
    dataset = pd.read_csv(dirdataset+"/"+f)
    _fclasses = list(dataset["Class"])
    _values = list(dataset["Value"])

    _values = [np.mean(_values[x:x+kernel_size]) for x in np.arange(0,len(_values), kernel_stride)]
    _fclasses = [1 if 1 in _fclasses[x:x+kernel_size] else 0 for x in np.arange(0,len(_fclasses), kernel_stride)]

    min_values = min(_values)
    max_values = max(_values)
    _values = [(v-min_values)/(max_values - min_values) for v in _values]
    _features = labelisation_automatic(_values, div=nbsplitlabel, epsilon=0.05)
    _ = _features.pop(0)
    _ = _fclasses.pop(0)
    _ = _values.pop(0)
    _ = _features.pop(-1)
    _ = _fclasses.pop(-1)
    _ = _values.pop(-1)

    nb_observation += len( list(dataset["Class"]) )

    _labels = [x for i, x in enumerate(_features) if i == _features.index(x)]
    _uclasses = [x for i, x in enumerate(_fclasses) if i == _fclasses.index(x)]
    _nlabels = len(_labels)

    _features = [_features[x:x+window] for x in np.arange(0,len(_features)-window+1, step)]
    _fclasses = [_fclasses[x:x+window] for x in np.arange(0,len(_fclasses)-window+1, step)]
    _values = [_values[x:x+window] for x in np.arange(0,len(_values)-window+1, step)]

    _classes = [0 for _ in range(len(_fclasses))]

    for i, f in enumerate(_fclasses):

        uniqueclasses = [x for i, x in enumerate(f) if x != 0]
        uniqueclasses = [x for i, x in enumerate(uniqueclasses) if i==uniqueclasses.index(x)]

        if not len(uniqueclasses) == 0:
            if len(uniqueclasses) == 1:
                _classes[i] = uniqueclasses[0]
            else:
                _classes[i] = uniqueclasses

    features_to_keep = _features
    fclasses_to_keep = _fclasses
    classes_to_keep = _classes
    labels_to_keep = _labels
    values_to_keep = _values


    features = features + features_to_keep
    fclasses = fclasses + fclasses_to_keep
    classes = classes + classes_to_keep
    values = values + values_to_keep
    labels = labels + labels_to_keep

labels = [x for i, x in enumerate(labels) if i == labels.index(x)]
nclasses = len([x for i, x in enumerate(classes) if i == classes.index(x)])
uclasses = [x for i, x in enumerate(classes) if i == classes.index(x)]


In [3]:
compotree = composition_tree(nclasses, iteration_max=100000000, epsilon=1e-6)
compotree.fit(features, classes)


In [26]:
with open('CDT_RCIS_SGE_w5.pickle', 'wb') as f:
    pickle.dump(compotree, f)

In [25]:
rpc, ACC, recall, precision, f1, inter = compotree.anomaly_rules()
print(ACC, recall, precision, f1, inter)
for i, rules in enumerate(rpc):
    #print(rules)
    if i !=0:
        for j, rule_branch in enumerate(rules):
            for k, r in enumerate(rule_branch):
                #if r.activated:
                print(r)
                #print( r.classes.count(1), r.split_rule["conditions"], r.split_rule["features"])
            print()

0.9973715448533153 0.8115501519756839 1.0 0.8959731543624161 0.39909645757921625
{'split_rule': {'composition': ['PN_100.0_50.0', 'DCstP_50.0_0'], 'condition': True, 'active': True}, 'classes': [0, 128], 'activated': True}

{'split_rule': {'composition': ['FCstP_0_50.0', 'PP_50.0_100.0'], 'condition': True, 'active': True}, 'classes': [0, 55], 'activated': True}
{'split_rule': {'composition': ['PN_100.0_50.0', 'DCstP_50.0_0'], 'condition': False, 'active': True}, 'classes': [23259, 201], 'activated': False}

{'split_rule': {'composition': ['FCstN_0_100.0', 'PN_100.0_100.0'], 'condition': True, 'active': True}, 'classes': [0, 24], 'activated': True}
{'split_rule': {'composition': ['FCstP_0_50.0', 'PP_50.0_100.0'], 'condition': False, 'active': True}, 'classes': [23259, 146], 'activated': False}
{'split_rule': {'composition': ['PN_100.0_50.0', 'DCstP_50.0_0'], 'condition': False, 'active': True}, 'classes': [23259, 201], 'activated': False}

{'split_rule': {'composition': ['FCstN_0_50.0'

In [15]:
kernel_size = 1
kernel_stride = 1
nbsplitlabel = 4
window = 5
dirdataset = "./campagne_sge/Test_DataSet/"
list_file = [f for f in listdir(dirdataset) if isfile(join(dirdataset, f))]
step = 1
offset = 1
mind = (window-1)/2

features = []
classes = []
fclasses = []
values = []
labels = []
nb_observation = 0

for i, f in enumerate(list_file):
    dataset = pd.read_csv(dirdataset+"/"+f)
    _fclasses = list(dataset["Class"])
    _values = list(dataset["Value"])

    _values = [np.mean(_values[x:x+kernel_size]) for x in np.arange(0,len(_values), kernel_stride)]
    _fclasses = [1 if 1 in _fclasses[x:x+kernel_size] else 0 for x in np.arange(0,len(_fclasses), kernel_stride)]

    min_values = min(_values)
    max_values = max(_values)
    _values = [(v-min_values)/(max_values - min_values) for v in _values]
    _features = labelisation_automatic(_values, div=nbsplitlabel, epsilon=0.05)
    _ = _features.pop(0)
    _ = _fclasses.pop(0)
    _ = _values.pop(0)
    _ = _features.pop(-1)
    _ = _fclasses.pop(-1)
    _ = _values.pop(-1)

    nb_observation += len( list(dataset["Class"]) )

    _labels = [x for i, x in enumerate(_features) if i == _features.index(x)]
    _uclasses = [x for i, x in enumerate(_fclasses) if i == _fclasses.index(x)]
    _nlabels = len(_labels)

    _features = [_features[x:x+window] for x in np.arange(0,len(_features)-window+1, step)]
    _fclasses = [_fclasses[x:x+window] for x in np.arange(0,len(_fclasses)-window+1, step)]
    _values = [_values[x:x+window] for x in np.arange(0,len(_values)-window+1, step)]

    _classes = [0 for _ in range(len(_fclasses))]

    for i, f in enumerate(_fclasses):

        uniqueclasses = [x for i, x in enumerate(f) if x != 0]
        uniqueclasses = [x for i, x in enumerate(uniqueclasses) if i==uniqueclasses.index(x)]

        if not len(uniqueclasses) == 0:
            if len(uniqueclasses) == 1:
                _classes[i] = uniqueclasses[0]
            else:
                _classes[i] = uniqueclasses

    features_to_keep = _features
    fclasses_to_keep = _fclasses
    classes_to_keep = _classes
    labels_to_keep = _labels
    values_to_keep = _values


    features = features + features_to_keep
    fclasses = fclasses + fclasses_to_keep
    classes = classes + classes_to_keep
    values = values + values_to_keep
    labels = labels + labels_to_keep

labels = [x for i, x in enumerate(labels) if i == labels.index(x)]
nclasses = len([x for i, x in enumerate(classes) if i == classes.index(x)])
uclasses = [x for i, x in enumerate(classes) if i == classes.index(x)]
print(len(features))

9798


In [19]:
TP, TN, FP, FN = 0, 0, 0, 0
for classe, observation in zip(classes, features):
    classe_found = compotree.which_leaf(observation)[1]
    if classe == 1 and classe_found == 1: TP+=1
    if classe == 0 and classe_found == 1: FP+=1
    if classe == 0 and classe_found == 0: TN+=1
    if classe == 1 and classe_found == 0: FN+=1

recall = TP/(TP+FN)
precision = TP/(TP+FP)
f1 = (2*recall*precision ) / (recall+precision)
acc = (TN+TP)/(TP+TN+FP+FN)
print(acc, precision, recall, f1)

0.9977546438048581 1.0 0.8135593220338984 0.897196261682243


In [ ]:
0.9973715448533153 0.8115501519756839 1.0 0.8959731543624161